In [1]:
# Import
import numpy as np
import time
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPRegressor
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt

In [8]:
# Select file
fol = '../offline_data/mlwwcoast_prep_data'
myvar = 'u'
bb = 0 
back = 24
forward = 24
fload = '{:s}/input_{:s}_buoy{:d}_back{:d}_for{:d}.npz'.format(fol,myvar,bb,back,forward)
D = np.load(fload)

# Keys in D
# X -> History Matrix (time,hr-back) where hr-back = 0,1,2,...
# Y -> Labels, (time,hr-forward) where hr_forward = 1,2,3... (pred we are trying to make)
# F -> Weather Forecast, (time,hr-forward)
# t -> time (time,)
# avg -> daily average, smoothed (time,1)

# stl = STL(co2, seasonal=13)
# res = stl.fit()
# fig = res.plot()

In [14]:
# Calculate residual
Y = D['Y']
X = D['X']
avg = D['avg']
F = D['F'][:,:23]
r = Y - F - avg
r.shape


In [70]:
# Loop over
# 1. buoy
# 2. forecast hour
# What to show relative skill to weather prediction

def test_model(model, X, y):
    x_tr, x_te, y_tr, y_te = train_test_split(X, y, test_size = 0.25, random_state = 42)
    model.fit(x_tr, y_tr)
    y_p = model.predict(x_te)
    y_tr_p = model.predict(x_tr)
    rmse_te = np.sqrt(np.mean(np.power(y_p-y_te,2)))
    rmse_tr = np.sqrt(np.mean(np.power(y_tr_p-y_tr,2)))
    return (rmse_te,rmse_tr)

# Set path to save data folder
fol = '../offline_data/mlwwcoast_prep_data'
fol_model = '../offline_data/mlwwcoast_models'

# How many buoys/hours to do
do_buoys = range(21)
do_hours = range(12)
Nb = len(do_buoys)
Nh = len(do_hours)

# Save rmse scores
lr_rmse_te = np.zeros((Nb))
lr_rmse_tr = np.zeros((Nb))
mr_rmse_te = np.zeros((Nb))
mr_rmse_tr = np.zeros((Nb))

ref_rmse_te = np.zeros(Nb)
ref_rmse_tr = np.zeros(Nb)

# Loop
start_time = time.time()
for bb in [0]:#do_buoys:
    
    print('Working on buoy {:d}',bb)

    # Select with variable to predict
    myvar = 'v' 

    # Data load parameters
    back = 24 # Select how many hours to go back in time
    forward = 24 # Select forecast hour to predict

    # Select file
    fload = '{:s}/input_{:s}_buoy{:d}_back{:d}_for{:d}.npz'.format(fol,myvar,bb,back,forward)
    D = np.load(fload)

    # Keys in D
    # X -> History Matrix (time,hr-back) where hr-back = 0,1,2,...
    # Y -> Labels, (time,hr-forward) where hr_forward = 1,2,3... (pred we are trying to make)
    # F -> Weather Forecast, (time,hr-forward)
    # t -> time (time,)
    # avg -> daily average, smoothed (time,1)

    # Regression Settings
    hr_for = 23
    hr_back = 24

    # Make input
    X = D['X'][:,1:hr_back]
    F = D['F'][:,:hr_for]
    Y = D['Y'][:,:hr_for]

    X = np.concatenate((X,D['avg']),axis=1)
    X = np.concatenate((X,F),axis=1)

    #X = X - D['avg'] # Removing yearly average
    Yr = Y - F # Residual of forecast
    
    # Train/Test Set
    x_tr, x_te, yr_tr, yr_te, y_tr, y_te, f_tr, f_te = train_test_split(X, Yr, Y, F, test_size = 0.25, random_state = 42)

    # Skill of numerical weather model
    ref_rmse_te[bb] = np.sqrt(np.mean(np.power(y_te[:,0]-x_te[:,0],2)))
    ref_rmse_tr[bb] = np.sqrt(np.mean(np.power(y_tr[:,0]-x_tr[:,0],2)))

    # Train models
    model = LinearRegression()
    (lr_rmse_te[bb],lr_rmse_tr[bb]) = test_model(model, X, Yr)
    # save the model to disk
    fname = '{:s}/lr_{:s}_buoy{:d}_back{:d}_for{:d}'.format(fol_model,myvar,bb,back,forward)
    pickle.dump(model, open(fname, 'wb'))
    
    # Convert from residual to forecast corrected
    Fc = f_te + model.predict(x_te)
    rmse_fc = np.sqrt(np.mean(np.power(Fc-y_te,2), axis=0)) #Forecast correction
    rmse_bl = np.sqrt(np.mean(np.power(f_te-y_te,2))) #Weather model baseline

    
    model = LinearRegression()
    (lr_rmse_te,lr_rmse_tr) = test_model(model, X, Y)
    Fc = model.predict(x_te)
    rmse_fc = np.sqrt(np.mean(np.power(Fc-y_te,2), axis=0)) #Forecast correction
#     model = MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=25)
#     (mr_rmse_te[bb,ff],mr_rmse_tr[bb,ff]) = test_model(model, X, y)
#     # save the model to disk
#     fname = '{:s}/mlp_{:s}_buoy{:d}_back{:d}_for{:d}'.format(fol_model,myvar,bb,back,forward)
#     pickle.dump(model, open(fname, 'wb'))

end_time = time.time()
print('Time elapsed: {:4.1f} seconds'.format(end_time-start_time))

# Save rmse's
# fname = '../offline_data/mlwwcoast_outputs/20220116_{:s}_rmse'.format(myvar)
# np.savez(fname, lr_te=lr_rmse_te, lr_tr=lr_rmse_tr, mr_tr=mr_rmse_tr, mr_te = mr_rmse_te,
#         buoys=do_buoys, hours=do_hours,
#         ref_te=ref_rmse_te, ref_tr=ref_rmse_tr)

print([rmse_fc,ref_rmse_te[bb]] )

Working on buoy {:d} 0
Time elapsed:  0.3 seconds
[array([1.83543626, 2.01692049, 2.10741502, 2.15402029, 2.20767491,
       2.20866818, 2.24760401, 2.25702282, 2.27207848, 2.28777436,
       2.28768177, 2.2980259 , 2.30218989, 2.29592222, 2.30337634,
       2.28618269, 2.30528771, 2.28782506, 2.28935386, 2.29924432,
       2.29652434, 2.297963  , 2.29394752]), 2.0829771892269315]


In [4]:
# Save rmse's
fname = '../offline_data/mlwwcoast_outputs/20211128_{:s}_rmse'.format(myvar)
np.savez(fname, lr_te=lr_rmse_te, lr_tr=lr_rmse_tr, mr_tr=mr_rmse_tr, mr_te = mr_rmse_te,
        buoys=do_buoys, hours=do_hours,
        ref_te=ref_rmse_te, ref_tr=ref_rmse_tr)

## BELOW IS EXTRA, stop here

In [15]:
class SelectHours(BaseEstimator, TransformerMixin):
    def __init__(self, hr_back=24):        
        self.hr_back = hr_back
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        return X[:,1:hr_back-1]
        

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

my_pipeline = Pipeline([
    ('selecthours',SelectHours(hr_back=24)),
    #('std_scaler',StandardScaler()),
])

X_prepared = my_pipeline.fit_transform(X)

LR = LinearRegression()


In [22]:
X.shape
y.shape

(6302,)

In [30]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LR, X, y, scoring="neg_mean_squared_error", cv=10)
lr_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print('Scores:', scores)
    print('Mean:',scores.mean())
    print('Std:',scores.std())
    
display_scores(lr_rmse_scores)

Scores: [1.43843141 1.42331392 1.13114424 1.85759804 1.68825704 1.2717601
 1.73177701 1.94795364 1.02712141 1.50835131]
Mean: 1.502570812794681
Std: 0.2896786849242856


In [41]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100,min_samples_split=100)
forest_reg.fit(X,y)


RandomForestRegressor(min_samples_split=100)

In [42]:
y_p = forest_reg.predict(X)
rmse = np.sqrt(np.mean(np.power(y_p-y,2)))
print(rmse)

1.2622097237533456


In [43]:
scores = cross_val_score(forest_reg, X, y, scoring="neg_mean_squared_error", cv=5)

In [44]:
lr_rmse_scores = np.sqrt(-scores)
display_scores(lr_rmse_scores)

Scores: [1.50583425 1.56661999 1.52521674 1.92240661 1.30423861]
Mean: 1.5648632396390894
Std: 0.20041048609207965


### Try neural net

In [45]:
from sklearn.neural_network import MLPRegressor

In [58]:
x_tr, x_te, y_tr, y_te = train_test_split(X, y, test_size = 0.25, random_state = 42)
reg_mlp = MLPRegressor(random_state=1, max_iter=500, hidden_layer_sizes=25)
reg_mlp.fit(x_tr,y_tr)

MLPRegressor(hidden_layer_sizes=25, max_iter=500, random_state=1)

In [59]:
y_p = reg_mlp.predict(x_te)
rmse = np.sqrt(np.mean(np.power(y_p-y_te,2)))
print(rmse)

1.5114376161412741


In [60]:
y_p = reg_mlp.predict(x_tr)
rmse = np.sqrt(np.mean(np.power(y_p-y_tr,2)))
print(rmse)

1.4430444371323001
